# テキストデータのクラスタリング結果の可視化 (3)

ワードクラウドを用いて可視化。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from wordcloud import WordCloud

# フィードデータの読み込み
feeds = pd.read_csv('data/output_en.csv')

# title と summary を結合して text 列を作成
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='. ', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

### 英語テキストに対する前処理・テキストのベクトル化

In [ ]:
symbols_to_remove = r'["`,.' + r"'" + r']'
stop_words = nltk.corpus.stopwords.words('english')
stop_words += ["'", '"', ':', ';', '.', ',', '-', '!', '?', "'s", '`', '•', '%']
stop_words += ['–', '—', '‘', '’', '“', '”', '…', '|', '#', '$', '&', "''", '(', ')']
stemmer = nltk.stem.porter.PorterStemmer()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

# 品詞の名称を変換
def wordnet_tag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    return None

def preprocess(text):
    tokens = []
    # 品詞のタグ付けをした各トークンについて
    for t in nltk.pos_tag(nltk.tokenize.word_tokenize(text.replace('-', ' '))):
        # 小文字化
        t0 = t[0].lower()
        # 不要な文字の削除
        t0 = re.sub(symbols_to_remove, '', t0)
        # 空文字列になったら次へ
        if t0 == '':
            continue
        # stop_words に含まれていないトークンのみを残す
        if t0 in stop_words:
            continue
        # カンマ区切りが入った数値からカンマを削除
        if t[1] == 'CD':
            t0 = t0.replace(',', '')
        # 見出し語化
        tag = wordnet_tag(t[1])
        if tag is None:
            t0 = lemmatizer.lemmatize(t0)
        else:
            t0 = lemmatizer.lemmatize(t0, tag)
        # ステミング
        t0 = stemmer.stem(t0)
        # リストに追加
        tokens.append(t0)
    # トークンのリストを返す
    return tokens

# BoW でベクトル化
vectorizer_bow = CountVectorizer(tokenizer=preprocess)
vector_bow = vectorizer_bow.fit_transform(df.text)

# TF-IDF でベクトル化
vectorizer_tfidf = TfidfVectorizer(tokenizer=preprocess)
vector_tfidf = vectorizer_tfidf.fit_transform(df.text)

### 非階層的クラスタリング

In [ ]:
# クラスタ数
N_clusters = 23

# KMeans の初期化
clusters = KMeans(n_clusters=N_clusters).fit_predict(vector_tfidf)

# 結果を DataFrame にまとめる
df_cluster = pd.DataFrame(clusters, columns=['cluster'])

### ワードクラウドの作成

In [ ]:
# 単語の DataFrame (BoW)
df_words = pd.DataFrame(vectorizer_bow.get_feature_names_out())

# テキストを BoW ベクトル化した結果を DataFrame
df_vector = pd.DataFrame.sparse.from_spmatrix(vector_bow)

# クラスタ 0 について、単語ごとの頻度を計算
_cdf = df_cluster[df_cluster['cluster'] == 0]
df_counts = pd.concat([df_words, df_vector.iloc[_cdf.index].sum()], axis=1)
df_counts.columns=['word', 'counts']

# 頻度 counts で降順でソートし、多く現れる単語 10 個を確認
df_counts.sort_values('counts', ascending=False).head(10)

In [ ]:
# dict 型に変換
freq = df_counts.set_index('word').to_dict()['counts']
# ワードクラウドの作成
wordcloud = WordCloud(background_color='white').fit_words(freq)
# 軸を消す
plt.axis('off')
# 描画
plt.imshow(wordcloud)